In [ ]:
from collections import defaultdict
from glob import glob
import os
import pandas as pd
import statistics as stats
import numpy as np
import math
import random
import matplotlib.pyplot as plt
from matplotlib import cm
import itertools

In [ ]:
abbrev = {'directed_forgetting_condition':'DF',
#           'directed_condition': 'DF',
         'flanker_condition':'FLANKER',
         'go_nogo_condition':'GNG',
         'n_back_condition':'NBACK',
          'delay_condition':'DELAY',
         'predictable_condition':'PREDICT',
#           'predictive_condition':'PREDICT',
         'shape_matching_condition':'SHAPE',
         'stop_signal_condition':'SS',
         'cue-task_condition':'TASK',
         'cue_condition':'CUE',
         'cued_condition':'CUE'}

In [ ]:
explore_id=['A3QAHF4UUBM7ZO',
'AIQT0DPRTXYYD',
'A3V1ZYXBJYCIIE',
'A2WYCY1FMQOD5F',
'A2S4YDJ9UGAXFQ',
'A3G55RJTW3BSGM',
'A1OSRAPSRT934Z',
'AVJUIF9QHQRY8',
'A1VCAMP3XM62R4',
'A3IW9415ZOO0EX',
'A55CXM7QR7R0N',
'A2XUADP5L61HQ5',
'AQL960O0LTRI8',
'A1DS5O8MSI3ZH0',
'A2YTO4EY3MNYAJ',
'AY7WPVKHVNBLG',
'A2OFN0A5CPLH57',
'A2JI5RNPPXE8QE',
'A1TS2SKXPX7ZED',
'A1YC558J4E5KZ',
'A2DWPP1KKAY0HG',
'AD7CUW86FWEKT',
'A2SIKW18T2DYYW',
'AQJWO4YPR3LUQ',
'A1L1SQ488YCCFJ',
'A3ISDIYTS02E8C',
'A3QJJR5Y3XE92N',
'A1R0689JPSQ3OF',
'A37EV8RZ82WT8E',
'A3GEHH49HNJM57',
'ASTR3EPUOKEXV',
'AVMIXXCHPD291',
'A2581F7TDPAMBQ',]

In [ ]:
### check for inconsistent naming among tasks and missing of necessary columns.
raw_dir = '/Users/tonyb/Desktop/all_data/'
if not os.path.exists(raw_dir + 'processed'):
    os.makedirs(raw_dir + 'processed')

task_dfs = defaultdict(pd.DataFrame)
predictive = []
directed = []
currenttrial = []
task = []
for subj_file in glob(raw_dir + 'A*/*'):
    filey = os.path.basename(subj_file) #filey is now cued_task_switching_with_directed_forgetting_A2YBZFSUD5OP7W.csv or any other subid_taskid.csv
    if filey.split('_')[-1].split('.')[0] in explore_id:
        cleaned_file_name = '_cleaned.'.join(filey.split('.')) # cleaned_file_name = 's061_ANT_cleaned.csv'
        cleaned_file_path = os.path.join(raw_dir + 'processed', cleaned_file_name) #os.path.join() adds exactly one directory separator (/) b/w  raw_dir + 'processed and cleaned_file_name
        df = pd.read_csv(subj_file)
        if 'exp_id' in df.columns:
            exp_id = df.exp_id.unique()[0] #for the network_attack battery, all rows in the exp_id column has the same value
        else:
            exp_id = '_'.join(os.path.basename(subj_file).split('_')[0:-1]).rstrip('.csv')
        if 'predictive_condition' in df.columns:
            predictive.append(exp_id)
        if 'predictive_dimension' in df.columns:
            predictive.append(exp_id)
        if 'directed_condition' in df.columns:
            directed.append(exp_id)
        if 'current_trial' not in df.columns and 'trial_index' in df.columns:
            currenttrial.append(exp_id)
        if 'task_condition' in df.columns:
            task.append(exp_id)

In [ ]:
set(task)

### Generate concat data from raw data

In [ ]:
raw_dir = '/Users/tonyb/Desktop/all_data/all_demographics/'
if not os.path.exists(raw_dir + 'processed'):
    os.makedirs(raw_dir + 'processed')

task_dfs = defaultdict(pd.DataFrame)
for subj_file in glob(raw_dir + '*demographics*'):
    filey = os.path.basename(subj_file) #filey is now cued_task_switching_with_directed_forgetting_A2YBZFSUD5OP7W.csv or any other taskid_subid.csv
#     if filey.split('_')[-1].split('.')[0] in explore_id:
    cleaned_file_name = '_cleaned.'.join(filey.split('.')) # cleaned_file_name = 's061_ANT_cleaned.csv'
    cleaned_file_path = os.path.join(raw_dir + 'processed', cleaned_file_name) #os.path.join() adds exactly one directory separator (/) b/w  raw_dir + 'processed and cleaned_file_name
    df = pd.read_csv(subj_file)
    if 'exp_id' in df.columns:
        exp_id = df.exp_id.unique()[0] #for the network_attack battery, all rows in the exp_id column has the same value
    else:
        exp_id = '_'.join(os.path.basename(subj_file).split('_')[0:-1]).rstrip('.csv')
    if 'worker_id' not in df.columns:
        df.loc[:,'worker_id'] = filey.split('_')[-1].split('.')[0] #filey.split('_')[-1] returns 'A2581F7TDPAMBQ.csv'; filey.split('_')[-1].split('.')[0] returns 'A2581F7TDPAMBQ'
    # really avoid any malfunctioning correct_trial
    if 'correct_trial' in df.columns and 'key_press'in df.columns and 'correct_response' in df.columns:
        df.rename(columns={'correct_trial':'original_correcttrial'},inplace=True)
        df['correct_trial'] = np.where(df['key_press']==df['correct_response'],1,0)
    # post process data, drop rows, etc.....
    for colname in df.columns:
        if 'predictive_condition' in colname:
            df.rename(columns={colname:'predictable_condition'},inplace=True)
            print('predictive_condition' + ' in df.columns; ' + ' ' + filey)
        if 'predictive_dimension' in colname:
            df.rename(columns={colname:'predictable_dimension'},inplace=True)
            print('predictive_dimension' + ' in df.columns; ' + ' ' + filey)
        if 'directed_condition' in colname:
            df.rename(columns={colname:'directed_forgetting_condition'},inplace=True)
            print('directed_condition' + ' in df.columns; ' + ' ' + filey)
        if 'task_condition' in colname:
            df.rename(columns={colname:'cue-task_condition'},inplace=True)
        if ('SS' not in colname) and ('delay' in colname):
            df.rename(columns={colname:'delay_condition'},inplace=True)
#                                    \'n_back_condition':'n_back_trial_type'},inplace=True)
    drop_columns = ['Unnamed: 0','final_accuracy','final_avg_rt', 'final_credit_var','final_missed_percent',
                    'final_responses_ok','internal_node_id','responses','stimulus',
                    'text','trial_index','trial_type','view_history']
    df.drop(drop_columns, axis=1, inplace=True, errors='ignore')
    keep_dict = {'trial_id': ['test_trial']}
    if 'trial_id' in df.columns:
        for row, vals in keep_dict.items():
            df = df.query('%s in %s' % (row, vals))
    if 'predictable' in exp_id or 'predictive' in exp_id:
        if 'current_trial' in df.columns:
            df = df.query('current_trial > 1') #remove first trial for all predictable-task switching tasks
        elif 'current_trial' not in df.columns:
            if 'cue-task_condition' in df.columns:
                df = df[-((pd.isna(df['cue-task_condition'])) | (df['cue-task_condition']=="na"))]
            elif 'predictable_condition' in df.columns:
                df = df[-((pd.isna(df['predictable_condition'])) | (df['predictable_condition']=="na"))]
            elif 'predictive_condition' in df.columns:
                df = df[-((pd.isna(df['predictive_condition'])) | (df['predictive_condition']=="na"))]
    if 'n_back' in exp_id and 'current_trial' in df.columns:
        df = df.query('current_trial > 3') #remove first 3 trials for all n_back tasks        
    if 'cue' in exp_id and 'cue-task_condition' in df.columns:
        df = df[-((pd.isna(df['cue-task_condition'])) | (df['cue-task_condition']=="na"))] #remove first trial, 'na' or null, for all cued-task switching tasks
#     df.to_csv(cleaned_file_path, index=False)
    task_dfs[exp_id] = pd.concat([task_dfs[exp_id], df], axis=0)

# if not os.path.exists(raw_dir + 'concat_data/explore'):
#     os.makedirs(raw_dir + 'concat_data/explore')
for task,df in task_dfs.items():
    if 'demographics' not in task:
        df.to_csv(raw_dir + 'concat_data/explore/%s.csv' % task, index=False)

tasks with no 'current_trial' column:
1. demographics_survey__stanford__nih
2. directed_forgetting_single_task_network
3. shape_matching_with_cued_task_switching
4. shape_matching_with_predictable_task_switching
5. shape_matching_single_task_network
6. cued_task_switching_single_task_network
7. go_nogo_single_task_network

In [ ]:
### check which tasks had malfunctioning correct_trial (that's automated by javascript)
concat_dir = '/Users/tonyb/Desktop/all_data/'

all_conditions=[]
all_workers=[]
for concat_file in glob(concat_dir + 'concat_data/explore/*'):
    filey = os.path.basename(concat_file) #filey is now cued_task_switching_with_directed_forgetting.csv or any other subid_taskid.csv
    concat_df = pd.read_csv(concat_file)
    if 'correct_trial' in df.columns and 'original_correcttrial'in df.columns:
        new_mean = stats.mean(concat_df['correct_trial'])
        original_mean = stats.mean(concat_df['original_correcttrial'])
        if new_mean != original_mean:
            print(filey)

In [ ]:
concat_dir = '/Users/tonyb/Desktop/all_data/'
taskDict = {}
for concat_file in glob(concat_dir + 'concat_data/explore/*'):
    filey = os.path.basename(concat_file) #filey is now cued_task_switching_with_directed_forgetting_A2YBZFSUD5OP7W.csv or any other taskid_subid.csv
    concat_df = pd.read_csv(concat_file)
    if 'exp_id' in concat_df.columns:
        exp_id = concat_df.exp_id.unique()[0] #for the network_attack battery, all rows in the exp_id column has the same value
    else:
        print(filey + " doesn't have an exp_id column")
#     print(exp_id)
    # collapsing across H and F in flanker_condition
    if ('flanker' in exp_id) and (len(concat_df['flanker_condition'].unique())>2):
        concat_df = concat_df.replace({'flanker_condition':[r'(^.*inc.*$)',r'(^.*_con.*$)']},\
                          {'flanker_condition':['incongruent','congruent']},regex=True)
    # collapsing 7 levels in shape_matching_condition: noise-response-incompatible (i.e. distractor != target, or D in second letter) vs. no-noise (i.e. no distractor, or N in second letter); also, disregarding SSS & DSD
    if ('shape' in exp_id) and (len(concat_df['shape_matching_condition'].unique())>2):
        concat_df = concat_df.replace({'shape_matching_condition':['SDD','DDS','DDD']},\
                          {'shape_matching_condition':['T!=D','T!=D','T!=D']})
        concat_df = concat_df.replace({'shape_matching_condition':['SNN','DNN']},\
                          {'shape_matching_condition':['NoD','NoD']})
    conditions = [i for i in concat_df.columns if ('condition' in i) and ('n_back_condition' not in i)]
    conditions.sort()
    for cond in conditions:
        values = concat_df[cond].unique()
        values.sort()
        taskDict.update({abbrev[cond]:values})

In [ ]:
taskDict

### Generate descriptive stats (qa files) from concat data

In [ ]:
concat_dir = '/Users/tonyb/Desktop/all_data/'

all_conditions=[]
all_workers=[]
test = []
for concat_file in glob(concat_dir + 'concat_data/explore/*'):
    filey = os.path.basename(concat_file) #filey is now cued_task_switching_with_directed_forgetting_A2YBZFSUD5OP7W.csv or any other taskid_subid.csv
    concat_df = pd.read_csv(concat_file)
    if 'exp_id' in concat_df.columns:
        exp_id = concat_df.exp_id.unique()[0] #for the network_attack battery, all rows in the exp_id column has the same value
    else:
        print(filey + " doesn't have an exp_id column")
    print(exp_id)
    if 'worker_id' in concat_df.columns and 'correct_trial' in concat_df.columns:
        if len(concat_df.worker_id.unique()) > 38:
            print(str(len(concat_df.worker_id.unique())) + ' id(s) in worker_id.')
        elif len(concat_df.correct_trial.unique()) > 2: 
            print('     ' + str(concat_df.correct_trial.unique()) + ' are unique values in correct_trial.')
    # collapsing across H and F in flanker_condition
    if ('flanker' in exp_id) and (len(concat_df['flanker_condition'].unique())>2):
        concat_df = concat_df.replace({'flanker_condition':[r'(^.*inc.*$)',r'(^.*_con.*$)']},\
                          {'flanker_condition':['incongruent','congruent']},regex=True)
    # collapsing 7 levels in shape_matching_condition: noise-response-incompatible (i.e. distractor != target, or D in second letter) vs. no-noise (i.e. no distractor, or N in second letter); also, disregarding SSS & DSD
    if ('shape' in exp_id) and (len(concat_df['shape_matching_condition'].unique())>2):
        concat_df = concat_df.replace({'shape_matching_condition':['SDD','DDS','DDD','D!=T']},\
                          {'shape_matching_condition':['DISTRACTOR','DISTRACTOR','DISTRACTOR','DISTRACTOR']})
        concat_df = concat_df.replace({'shape_matching_condition':['SNN','DNN','NoD']},\
                          {'shape_matching_condition':['CONTROL','CONTROL','CONTROL']})
    ###generating colnames for qa_df
    qa_colnames = []
#     conditions = [i for i in concat_df.columns if ('condition' in i)]
    conditions = [i for i in concat_df.columns if ('condition' in i) and ('n_back_condition' not in i)]
    conditions.sort()
    print(conditions)
#     for i in conditions:
#         if len(concat_df[i].unique()) >2: 
#             print(exp_id)
#             print(concat_df[i].unique())
    # in preparation for calculation of ssrt: replace and rank GoRT:
    if ('stop' in exp_id) and ('stop_signal_condition' in concat_df.columns): #only for stop-signal tasks
        concat_df['GoRTWithReplacement'] = np.where(concat_df['stop_signal_condition']=='go',np.where(concat_df['rt']== -1,2000,concat_df['rt']), np.nan)
        concat_df['GoRTWithReplacementRank'] = concat_df.groupby(['worker_id']+conditions, sort=False)['GoRTWithReplacement'].rank(ascending=True, method='first')
        concat_df.drop([i for i in concat_df.columns if 'Unnamed' in i], axis=1, inplace=True, errors='ignore')
        concat_df.to_csv(concat_file,index=False)
    
    
    ### Tasks with 1 condition variable
    if len(conditions) == 1:
        ordered_condition0 = concat_df[conditions[0]].unique()
        ordered_condition0.sort()
        for condition0 in ordered_condition0:
            condition_name = str(abbrev[conditions[0]] + ':' + str(condition0)) #TASK:stay
            all_conditions.append(condition_name)
            if 'SS:go' in condition_name:
                qa_colnames.append(str('countGoTrials_' + condition_name))
            elif 'SS:stop' in condition_name:
                for i in ['ssd_', 'ssrt_', 'm(stopfail_rt)_', 'sd(stopfail_rt)_', 'stopfail_rate_']:
                    qa_colname = str(i + condition_name)
                    qa_colnames.append(qa_colname)
            elif 'GNG:nogo' in condition_name:
                for i in ['m(nogofail_rt)_', 'sd(nogofail_rt)_', 'nogofail_rate_']:
                    qa_colname = str(i + condition_name)
                    qa_colnames.append(qa_colname)
            else:
                for i in ['m(rt)_','sd(rt)_','acc_']:
                    qa_colname = str(i + condition_name) #m(rt)_TASK:stay
                    qa_colnames.append(qa_colname)
        qa_df = pd.DataFrame(index=concat_df.worker_id.unique(),columns=qa_colnames)
        for condition0 in ordered_condition0:
            condition_name = str(abbrev[conditions[0]] + ':' + str(condition0))
            for worker_id in qa_df.index:
                if worker_id not in all_workers:
                    all_workers.append(worker_id)
                ### df by worker_id to calculate descriptive stats
                df_subj = concat_df[(concat_df['worker_id']==worker_id) & (concat_df[conditions[0]]==condition0)]
                ### number of go trials
                if ('stop' in exp_id) and ('SS:go' in condition_name):
                    countGoTrials = str('countGoTrials_'+condition_name)
                    qa_df.at[worker_id,countGoTrials] = len(df_subj) #len(df_subj): all Go trials 
                ### mean response times
                mrt = str('m(rt)_'+condition_name)
                if len(df_subj['rt'][(df_subj['correct_trial']==1)])==0:
#                     print(worker_id + ' had 0 correct_trial in ' + condition_name + ' in ' + exp_id)
                    qa_df.at[worker_id,mrt] = np.nan
                else:
                    mean_rt = stats.mean(df_subj['rt'][(df_subj['correct_trial']==1)])
                    qa_df.at[worker_id,mrt] = round(mean_rt,2)
                ### sd response times
                sdrt = str('sd(rt)_'+condition_name)
                if len(df_subj['rt'][(df_subj['correct_trial']==1)])<2:
#                     print(worker_id + ' had <2 correct_trial in ' + condition_name + ' in ' + exp_id)
                    qa_df.at[worker_id,sdrt] = np.nan
                else:
                    sd_rt = stats.stdev(df_subj['rt'][(df_subj['correct_trial']==1)])
                    qa_df.at[worker_id,sdrt] = round(sd_rt,2)
                ### accuracy rate, i.e., count of correct responses
                accuracy = len(df_subj[df_subj['correct_trial']==1])/len(df_subj)
                acc = str('acc_'+condition_name)
                qa_df.at[worker_id,acc] = round(accuracy,2)
                if ('_no' in exp_id) and (condition0 == 'nogo'):
                    if 'go_nogo_condition' in df_subj.columns:
                        df_subj_nogo = df_subj[(df_subj['go_nogo_condition']=='nogo')]
                        # stopfail rate
                        nogofail = len(df_subj_nogo[df_subj_nogo['correct_trial']==0])/len(df_subj_nogo)
                        nogo_fail = str('nogofail_rate_'+condition_name)
                        qa_df.at[worker_id,nogo_fail] = round(nogofail,2)
                        #m(rt_nogofail)
                        mrt_nogofail = str('m(nogofail_rt)_'+condition_name)    
                        if len(df_subj_nogo['rt'][(df_subj_nogo['correct_trial']==0)])==0:
#                             print(worker_id + ' had 0 nogofail trial in ' + condition_name + ' in ' + exp_id)
                            qa_df.at[worker_id,mrt_nogofail] = np.nan
                        else:
                            mrtfail = stats.mean(df_subj_nogo['rt'][(df_subj_nogo['correct_trial']==0)])
                            qa_df.at[worker_id,mrt_nogofail] = round(mrtfail,2)
                        #sd(rt_nogofail)
                        sdrt_nogofail = str('sd(nogofail_rt)_'+condition_name)
                        if len(df_subj_nogo['rt'][(df_subj_nogo['correct_trial']==0)])<2:
#                             print(worker_id + ' had <2 nogofail trial in ' + condition_name + ' in ' + exp_id)
                            qa_df.at[worker_id,sdrt_nogofail] = np.nan
                        else:
                            sdrtfail = stats.stdev(df_subj_nogo['rt'][(df_subj_nogo['correct_trial']==0)])
                            qa_df.at[worker_id,sdrt_nogofail] = round(sdrtfail,2)
                    else:
                        print('go_nogo_condition NOT in '+exp_id+'.columns')
                if ('stop' in exp_id) and (condition0 == 'stop'):
                    if 'stop_signal_condition' in df_subj.columns and 'SS_delay' in df_subj.columns:
                        df_subj_stop = df_subj[(df_subj['stop_signal_condition']=='stop')]
                        # ssd
                        ssd = stats.mean(df_subj_stop['SS_delay'])
                        SS_delay = str('ssd_'+condition_name)
                        qa_df.at[worker_id,SS_delay] = round(ssd,2)
                        # stopfail rate
                        stopfail = len(df_subj_stop[df_subj_stop['correct_trial']==0])/len(df_subj_stop)
                        stop_fail = str('stopfail_rate_'+condition_name)
                        qa_df.at[worker_id,stop_fail] = round(stopfail,2)
                        #ssrt
                        rank = int(qa_df.at[worker_id,[i for i in qa_df.columns if ('countGoTrials_' in i) & ('SS:go' in i)][0]]*stopfail)
                        nthRT = list(concat_df['GoRTWithReplacement'][(concat_df['worker_id']==worker_id) & (concat_df['GoRTWithReplacementRank']==rank)])[0]
                        ssrt = nthRT - ssd
                        SSRT = str('ssrt_'+condition_name)
                        qa_df.at[worker_id,SSRT] = round(ssrt,2)
                        #m(rt_stopfail)
                        mrt_stopfail = str('m(stopfail_rt)_'+condition_name)    
                        if len(df_subj_stop['rt'][(df_subj_stop['correct_trial']==0)])==0:
#                             print(worker_id + ' had 0 stopfail trial in ' + condition_name + ' in ' + exp_id)
                            qa_df.at[worker_id,mrt_stopfail] = np.nan
                        else:
                            mrtfail = stats.mean(df_subj_stop['rt'][(df_subj_stop['correct_trial']==0)])
                            qa_df.at[worker_id,mrt_stopfail] = round(mrtfail,2)
                        #sd(rt_stopfail)
                        sdrt_stopfail = str('sd(stopfail_rt)_'+condition_name)
                        if len(df_subj_stop['rt'][(df_subj_stop['correct_trial']==0)])<2:
#                             print(worker_id + ' had <2 stopfail trial in ' + condition_name + ' in ' + exp_id)
                            qa_df.at[worker_id,sdrt_stopfail] = np.nan
                        else:
                            sdrtfail = stats.stdev(df_subj_stop['rt'][(df_subj_stop['correct_trial']==0)])
                            qa_df.at[worker_id,sdrt_stopfail] = round(sdrtfail,2)
#                             #acc_stopfail
#                             accfail = round(len(df_subj_stop[df_subj_stop['correct_trial']==0])/len(df_subj_stop)*100,2)
#                             accrt_stopfail = str('acc(stopfail)_)_'+condition_name)
#                             qa_df.at[worker_id,accrt_stopfail] = accfail
                    else:
                        print('stop_signal_condition and SS_delay NOT in '+exp_id+'.columns')
    
    
    ### Tasks with 2 condition variables
    elif len(conditions) == 2:
        ordered_condition0 = concat_df[conditions[0]].unique()
        ordered_condition0.sort()
        ordered_condition1 = concat_df[conditions[1]].unique()
        ordered_condition1.sort()
        for condition0 in ordered_condition0:
            for condition1 in ordered_condition1:
                condition_name = str(abbrev[conditions[0]] + ':' + str(condition0) + '_&_' + abbrev[conditions[1]] + ':' + str(condition1)) #TASK:stay_&_GNG:go
                all_conditions.append(condition_name)
                if 'SS:go' in condition_name:
                    qa_colnames.append(str('countGoTrials_' + condition_name))
                elif 'SS:stop' in condition_name:
                    for i in ['ssd_', 'ssrt_', 'm(stopfail_rt)_', 'sd(stopfail_rt)_', 'stopfail_rate_']:
                        qa_colname = str(i + condition_name)
                        qa_colnames.append(qa_colname)
                elif 'GNG:nogo' in condition_name:
                    for i in ['m(nogofail_rt)_', 'sd(nogofail_rt)_', 'nogofail_rate_']:
                        qa_colname = str(i + condition_name)
                        qa_colnames.append(qa_colname)
                else:
                    for i in ['m(rt)_','sd(rt)_','acc_']:
                        qa_colname = str(i + condition_name) #m(rt)_TASK:stay
                        qa_colnames.append(qa_colname)
        qa_df = pd.DataFrame(index=concat_df.worker_id.unique(),columns=qa_colnames)
        for condition0 in ordered_condition0:
            for condition1 in ordered_condition1:
                if 'stop' not in conditions[0]:
                    no_stop_condition_name = str(abbrev[conditions[0]] + ':' + str(condition0))
                else:
                    print('stop_signal_condition NOT last in conditions')
                condition_name = str(abbrev[conditions[0]] + ':' + str(condition0) + '_&_' + abbrev[conditions[1]] + ':' + str(condition1)) #TASK:stay_&_GNG:go
                for worker_id in qa_df.index:
                    if worker_id not in all_workers:
                        all_workers.append(worker_id)
                    df_subj = concat_df[(concat_df['worker_id']==worker_id) & (concat_df[conditions[0]]==condition0) & (concat_df[conditions[1]]==condition1)]
                    # number of go trials
                    if ('stop' in exp_id) and ('SS:go' in condition_name):
                        countGoTrials = str('countGoTrials_'+condition_name)
                        qa_df.at[worker_id,countGoTrials] = len(df_subj)  #len(df_subj): all Go trials in one of the other condition's levels
                    # mean response times
                    mrt = str('m(rt)_'+condition_name)
                    if len(df_subj['rt'][(df_subj['correct_trial']==1)])==0:
#                         print(worker_id + ' had 0 correct trial in ' + condition_name + ' in ' + exp_id)
                        qa_df.at[worker_id,mrt] = np.nan
                    else:
                        mean_rt = round(stats.mean(df_subj['rt'][(df_subj['correct_trial']==1)]),2)
                        qa_df.at[worker_id,mrt] = mean_rt
                    # sd response times
                    sdrt = str('sd(rt)_'+condition_name)
                    if len(df_subj['rt'][(df_subj['correct_trial']==1)])<2:
#                         print(worker_id + ' had <2 correct_trial in ' + condition_name + ' in ' + exp_id)
                        qa_df.at[worker_id,sdrt] = np.nan
                    else:
                        sd_rt = round(stats.stdev(df_subj['rt'][(df_subj['correct_trial']==1)]),2)
                        qa_df.at[worker_id,sdrt] = sd_rt
                    # accuracy rate, i.e., count of correct responses
                    if len(df_subj) !=0:
                        accuracy = round(len(df_subj[df_subj['correct_trial']==1])/len(df_subj),2)
                        acc = str('acc_'+condition_name)
                        qa_df.at[worker_id,acc] = accuracy
#                     else:
#                         qa_df.at[worker_id,acc] = np.nan
                    if '_no' in exp_id and condition0 == 'nogo' or condition1 == 'nogo' and 'go_nogo_condition' in df_subj.columns:
                        df_subj_nogo = df_subj[(df_subj['go_nogo_condition']=='nogo')]
                        # nogofail rate
                        if len(df_subj_nogo) !=0:
                            nogofail = round(len(df_subj_nogo[df_subj_nogo['correct_trial']==0])/len(df_subj_nogo),2)
                            nogo_fail = str('nogofail_rate_'+condition_name)
                            qa_df.at[worker_id,nogo_fail] = nogofail
                        else:
                            qa_df.at[worker_id,nogo_fail] = np.nan
                        #m(rt_nogofail)
                        mrt_nogofail = str('m(nogofail_rt)_'+condition_name)
                        if len(df_subj_nogo['rt'][(df_subj_nogo['correct_trial']==0)])==0:
#                             print(worker_id + ' had 0 nogofail trial in ' + condition_name + ' in ' + exp_id)
                            qa_df.at[worker_id,mrt_nogofail] = np.nan
                        else:
                            mrtfail = round(stats.mean(df_subj_nogo['rt'][(df_subj_nogo['correct_trial']==0)]),2)
                            qa_df.at[worker_id,mrt_nogofail] = mrtfail
                        #sd(rt_nogofail)
                        sdrt_nogofail = str('sd(nogofail_rt)_'+condition_name)
                        if len(df_subj_nogo['rt'][(df_subj_nogo['correct_trial']==0)])<2:
#                             print(worker_id + ' had <2 nogofail trial in ' + condition_name + ' in ' + exp_id)
                            qa_df.at[worker_id,sdrt_nogofail] = np.nan
                        else:
                            sdrtfail = round(stats.stdev(df_subj_nogo['rt'][(df_subj_nogo['correct_trial']==0)]),2)
                            qa_df.at[worker_id,sdrt_nogofail] = sdrtfail
                    if 'stop' in exp_id and condition0 == 'stop' or condition1 == 'stop' and 'stop_signal_condition' in df_subj.columns and 'SS_delay' in df_subj.columns:
                        df_subj_stop = df_subj[(df_subj['stop_signal_condition']=='stop')]
                        # ssd
                        if len(df_subj_stop['SS_delay']) >1:
                            ssd = round(stats.mean(df_subj_stop['SS_delay']),2)
                            SS_delay = str('ssd_'+condition_name)
                            qa_df.at[worker_id,SS_delay] = ssd
                        else:
                            qa_df.at[worker_id,SS_delay] = np.nan
                        # stopfail rate
                        if len(df_subj_stop) !=0:
                            stopfail = round(len(df_subj_stop[df_subj_stop['correct_trial']==0])/len(df_subj_stop),2)
                            stop_fail = str('stopfail_rate_'+condition_name)
                            qa_df.at[worker_id,stop_fail] = stopfail
                        else:
                            qa_df.at[worker_id,stop_fail] = np.nan
                        #ssrt
                        SSRT = str('ssrt_'+condition_name)
                        rank = int(qa_df.at[worker_id,[i for i in qa_df.columns if ('countGoTrials_' in i) & (no_stop_condition_name in i) & ('SS:go' in i)][0]]*stopfail)
                        if rank >0:
                            nthRT = list(concat_df['GoRTWithReplacement'][(concat_df['worker_id']==worker_id) & (concat_df[conditions[0]]==condition0) & (concat_df['GoRTWithReplacementRank']==rank)])[0]
                            ssrt = nthRT - ssd
                            qa_df.at[worker_id,SSRT] = round(ssrt,2)
                        else:
                            qa_df.at[worker_id,SSRT] = 'rank <0'
                        #m(rt_stopfail)
                        mrt_stopfail = str('m(stopfail_rt)_'+condition_name)
                        if len(df_subj_stop['rt'][(df_subj_stop['correct_trial']==0)])==0:
#                             print(worker_id + ' had 0 stopfail trial in ' + condition_name + ' in ' + exp_id)
                            qa_df.at[worker_id,mrt_stopfail] = np.nan
                        else:
                            mrtfail = round(stats.mean(df_subj_stop['rt'][(df_subj_stop['correct_trial']==0)]),2)
                            qa_df.at[worker_id,mrt_stopfail] = mrtfail
                        #sd(rt_stopfail)
                        sdrt_stopfail = str('sd(stopfail_rt)_'+condition_name)
                        if len(df_subj_stop['rt'][(df_subj_stop['correct_trial']==0)])<2:
#                             print(worker_id + ' had <2 stopfail trial in ' + condition_name + ' in ' + exp_id)
                            qa_df.at[worker_id,sdrt_stopfail] = np.nan
                        else:
                            sdrtfail = round(stats.stdev(df_subj_stop['rt'][(df_subj_stop['correct_trial']==0)]),2)
                            qa_df.at[worker_id,sdrt_stopfail] = sdrtfail
#                             #acc_stopfail
#                             accfail = round(len(df_subj_stop[df_subj_stop['correct_trial']==0])/len(df_subj_stop)*100,2)
#                             accrt_stopfail = str('acc(stopfail)_)_'+condition_name)
#                             qa_df.at[worker_id,accrt_stopfail] = accfail
#                     else:
#                         print('stop_signal_condition and SS_delay NOT in '+exp_id+'.columns')


    ### DUAL-TASKs with 3 _condition variables
    elif len(conditions) == 3:
        ordered_condition0 = concat_df[conditions[0]].unique()
        ordered_condition0.sort()
        ordered_condition1 = concat_df[conditions[1]].unique()
        ordered_condition1.sort()
        ordered_condition2 = concat_df[conditions[2]].unique()
        ordered_condition2.sort()
        for condition0 in ordered_condition0:
            for condition1 in ordered_condition1:
                for condition2 in ordered_condition2:
                    condition_name = str(abbrev[conditions[0]] + ':' + str(condition0) + '_&_' + abbrev[conditions[1]] + ':' + str(condition1) + '_&_' + abbrev[conditions[2]] + ':' + str(condition2)) #TASK:stay_&_GNG:go
                    all_conditions.append(condition_name)
                    if 'SS:go' in condition_name:
                        qa_colnames.append(str('countGoTrials_' + condition_name))
                    elif 'SS:stop' in condition_name:
                        for i in ['ssd_', 'ssrt_', 'm(stopfail_rt)_', 'sd(stopfail_rt)_', 'stopfail_rate_']:
                            qa_colname = str(i + condition_name)
                            qa_colnames.append(qa_colname)
                    elif 'GNG:nogo' in condition_name:
                        for i in ['m(nogofail_rt)_', 'sd(nogofail_rt)_', 'nogofail_rate_']:
                            qa_colname = str(i + condition_name)
                            qa_colnames.append(qa_colname)
                    else:
                        for i in ['m(rt)_','sd(rt)_','acc_']:
                            qa_colname = str(i + condition_name) #m(rt)_TASK:stay
                            qa_colnames.append(qa_colname)
        qa_df = pd.DataFrame(index=concat_df.worker_id.unique(),columns=qa_colnames)
        for condition0 in ordered_condition0:
            for condition1 in ordered_condition1:
                for condition2 in ordered_condition2:
                    if ('stop' not in conditions[0]) and ('stop' not in conditions[1]):
                        no_stop_condition_name = str(abbrev[conditions[0]] + ':' + str(condition0) + '_&_' + abbrev[conditions[1]] + ':' + str(condition1))
                    else:
                        print('stop_signal_condition is NOT last in conditions')
#                         no_stop_condition_name = str(abbrev[conditions[1]] + ':' + str(condition1))
                    condition_name = str(abbrev[conditions[0]] + ':' + str(condition0) + '_&_' + abbrev[conditions[1]] + ':' + str(condition1) + '_&_' + abbrev[conditions[2]] + ':' + str(condition2)) #TASK:stay_&_GNG:go
                    
                    for worker_id in qa_df.index:
                        if worker_id not in all_workers:
                            all_workers.append(worker_id)
                        df_subj = concat_df[(concat_df['worker_id']==worker_id) & (concat_df[conditions[0]]==condition0) & (concat_df[conditions[1]]==condition1) & (concat_df[conditions[2]]==condition2)]
                        # number of go trials
                        if ('stop' in exp_id) and ('SS:go' in condition_name):
                            countGoTrials = str('countGoTrials_'+condition_name)
                            qa_df.at[worker_id,countGoTrials] = len(df_subj)  #len(df_subj): all Go trials in one of the other condition's levels
                        # mean response times
                        mrt = str('m(rt)_'+condition_name)
                        if len(df_subj['rt'][(df_subj['correct_trial']==1)])==0:
    #                         print(worker_id + ' had 0 correct trial in ' + condition_name + ' in ' + exp_id)
                            qa_df.at[worker_id,mrt] = np.nan
                        else:
                            mean_rt = round(stats.mean(df_subj['rt'][(df_subj['correct_trial']==1)]),2)
                            qa_df.at[worker_id,mrt] = mean_rt
                        # sd response times
                        sdrt = str('sd(rt)_'+condition_name)
                        if len(df_subj['rt'][(df_subj['correct_trial']==1)])<2:
    #                         print(worker_id + ' had <2 correct_trial in ' + condition_name + ' in ' + exp_id)
                            qa_df.at[worker_id,sdrt] = np.nan
                        else:
                            sd_rt = round(stats.stdev(df_subj['rt'][(df_subj['correct_trial']==1)]),2)
                            qa_df.at[worker_id,sdrt] = sd_rt
                        # accuracy rate, i.e., count of correct responses
                        if len(df_subj) !=0:
                            accuracy = round(len(df_subj[df_subj['correct_trial']==1])/len(df_subj),2)
                            acc = str('acc_'+condition_name)
                            qa_df.at[worker_id,acc] = accuracy
#                         else:
#                             qa_df.at[worker_id,acc] = np.nan
                        if '_no' in exp_id and condition0 == 'nogo' or condition1 == 'nogo' or condition2 == 'nogo' and 'go_nogo_condition' in df_subj.columns:
                            df_subj_nogo = df_subj[(df_subj['go_nogo_condition']=='nogo')]
                            # nogofail rate
                            if len(df_subj_nogo) !=0:
                                nogofail = round(len(df_subj_nogo[df_subj_nogo['correct_trial']==0])/len(df_subj_nogo),2)
                                nogo_fail = str('nogofail_rate_'+condition_name)
                                qa_df.at[worker_id,nogo_fail] = nogofail
#                             else:
#                                 qa_df.at[worker_id,nogo_fail] = np.nan
                            #m(rt_nogofail)
                            mrt_nogofail = str('m(nogofail_rt)_'+condition_name)
                            if len(df_subj_nogo['rt'][(df_subj_nogo['correct_trial']==0)])==0:
    #                             print(worker_id + ' had 0 nogofail trial in ' + condition_name + ' in ' + exp_id)
                                qa_df.at[worker_id,mrt_nogofail] = np.nan
                            else:
                                mrtfail = round(stats.mean(df_subj_nogo['rt'][(df_subj_nogo['correct_trial']==0)]),2)
                                qa_df.at[worker_id,mrt_nogofail] = mrtfail
                            #sd(rt_nogofail)
                            sdrt_nogofail = str('sd(nogofail_rt)_'+condition_name)
                            if len(df_subj_nogo['rt'][(df_subj_nogo['correct_trial']==0)])<2:
    #                             print(worker_id + ' had <2 nogofail trial in ' + condition_name + ' in ' + exp_id)
                                qa_df.at[worker_id,sdrt_nogofail] = np.nan
                            else:
                                sdrtfail = round(stats.stdev(df_subj_nogo['rt'][(df_subj_nogo['correct_trial']==0)]),2)
                                qa_df.at[worker_id,sdrt_nogofail] = sdrtfail
                        if 'stop' in exp_id and condition0 == 'stop' or condition1 == 'stop' or condition2 == 'stop' and 'stop_signal_condition' in df_subj.columns and 'SS_delay' in df_subj.columns:
                            df_subj_stop = df_subj[(df_subj['stop_signal_condition']=='stop')]
                            # ssd
                            if len(df_subj_stop['SS_delay']) >1:
                                ssd = round(stats.mean(df_subj_stop['SS_delay']),2)
                                SS_delay = str('ssd_'+condition_name)
                                qa_df.at[worker_id,SS_delay] = ssd
#                             else:
#                                 qa_df.at[worker_id,SS_delay] = len(df_subj_stop['SS_delay'])
                            # stopfail rate
                            if len(df_subj_stop) !=0:
                                stopfail = round(len(df_subj_stop[df_subj_stop['correct_trial']==0])/len(df_subj_stop),2)
                                stop_fail = str('stopfail_rate_'+condition_name)
                                qa_df.at[worker_id,stop_fail] = stopfail
#                             else:
#                                 qa_df.at[worker_id,stop_fail] = np.nan
                            #ssrt
                            SSRT = str('ssrt_'+condition_name)
                            rank = int(qa_df.at[worker_id,[i for i in qa_df.columns if ('countGoTrials_' in i) & (no_stop_condition_name in i) & ('SS:go' in i)][0]]*stopfail)
                            if rank >0:
                                nthRT = list(concat_df['GoRTWithReplacement'][(concat_df['worker_id']==worker_id) & (concat_df[conditions[0]]==condition0) & (concat_df[conditions[1]]==condition1) & (concat_df['GoRTWithReplacementRank']==rank)])[0]
                                ssrt = nthRT - ssd
                                qa_df.at[worker_id,SSRT] = round(ssrt,2)
                            else:
                                qa_df.at[worker_id,SSRT] = 'rank <0'
                            #m(rt_stopfail)
                            mrt_stopfail = str('m(stopfail_rt)_'+condition_name)
                            if len(df_subj_stop['rt'][(df_subj_stop['correct_trial']==0)])==0:
    #                             print(worker_id + ' had 0 stopfail trial in ' + condition_name + ' in ' + exp_id)
                                qa_df.at[worker_id,mrt_stopfail] = np.nan
                            else:
                                mrtfail = round(stats.mean(df_subj_stop['rt'][(df_subj_stop['correct_trial']==0)]),2)
                                qa_df.at[worker_id,mrt_stopfail] = mrtfail
                            #sd(rt_stopfail)
                            sdrt_stopfail = str('sd(stopfail_rt)_'+condition_name)
                            if len(df_subj_stop['rt'][(df_subj_stop['correct_trial']==0)])<2:
    #                             print(worker_id + ' had <2 stopfail trial in ' + condition_name + ' in ' + exp_id)
                                qa_df.at[worker_id,sdrt_stopfail] = np.nan
                            else:
                                sdrtfail = round(stats.stdev(df_subj_stop['rt'][(df_subj_stop['correct_trial']==0)]),2)
                                qa_df.at[worker_id,sdrt_stopfail] = sdrtfail
#                             #acc_stopfail
#                             accfail = round(len(df_subj_stop[df_subj_stop['correct_trial']==0])/len(df_subj_stop)*100,2)
#                             accrt_stopfail = str('acc(stopfail)_)_'+condition_name)
#                             qa_df.at[worker_id,accrt_stopfail] = accfail
    #                     else:
    #                         print('stop_signal_condition and SS_delay NOT in '+exp_id+'.columns')
    
    
        ### DUAL-TASKs with 4 _condition variables
    elif len(conditions) == 4:
        ordered_condition0 = concat_df[conditions[0]].unique()
        ordered_condition0.sort()
        ordered_condition1 = concat_df[conditions[1]].unique()
        ordered_condition1.sort()
        ordered_condition2 = concat_df[conditions[2]].unique()
        ordered_condition2.sort()
        ordered_condition3 = concat_df[conditions[3]].unique()
        ordered_condition3.sort()
        for condition0 in ordered_condition0:
            for condition1 in ordered_condition1:
                for condition2 in ordered_condition2:
                    for condition3 in ordered_condition3:
                        condition_name = str(abbrev[conditions[0]] + ':' + str(condition0) + '_&_' + abbrev[conditions[1]] + ':' + str(condition1) + '_&_' + abbrev[conditions[2]] + ':' + str(condition2) + '_&_' + abbrev[conditions[3]] + ':' + str(condition3)) #TASK:stay_&_GNG:go
                        all_conditions.append(condition_name)
                        if 'SS:go' in condition_name:
                            qa_colnames.append(str('countGoTrials_' + condition_name))
                        elif 'SS:stop' in condition_name:
                            for i in ['ssd_', 'ssrt_', 'm(stopfail_rt)_', 'sd(stopfail_rt)_', 'stopfail_rate_']:
                                qa_colname = str(i + condition_name)
                                qa_colnames.append(qa_colname)
                        elif 'GNG:nogo' in condition_name:
                            for i in ['m(nogofail_rt)_', 'sd(nogofail_rt)_', 'nogofail_rate_']:
                                qa_colname = str(i + condition_name)
                                qa_colnames.append(qa_colname)
                        else:
                            for i in ['m(rt)_','sd(rt)_','acc_']:
                                qa_colname = str(i + condition_name) #m(rt)_TASK:stay
                                qa_colnames.append(qa_colname)
        qa_df = pd.DataFrame(index=concat_df.worker_id.unique(),columns=qa_colnames)
        for condition0 in ordered_condition0:
            for condition1 in ordered_condition1:
                for condition2 in ordered_condition2:
                    for condition3 in ordered_condition3:
                        if ('stop' not in conditions[0]) and ('stop' not in conditions[1]) and ('stop' not in conditions[2]):
                            no_stop_condition_name = str(abbrev[conditions[0]] + ':' + str(condition0) + '_&_' + abbrev[conditions[1]] + ':' + str(condition1) + '_&_' + abbrev[conditions[2]] + ':' + str(condition2))
                        else:
                            print('stop_signal_condition is NOT last in conditions')
    #                         no_stop_condition_name = str(abbrev[conditions[1]] + ':' + str(condition1))
                        condition_name = str(abbrev[conditions[0]] + ':' + str(condition0) + '_&_' + abbrev[conditions[1]] + ':' + str(condition1) + '_&_' + abbrev[conditions[2]] + ':' + str(condition2) + '_&_' + abbrev[conditions[3]] + ':' + str(condition3)) #TASK:stay_&_GNG:go
                        for worker_id in qa_df.index:
                            if worker_id not in all_workers:
                                all_workers.append(worker_id)
                            df_subj = concat_df[(concat_df['worker_id']==worker_id) & (concat_df[conditions[0]]==condition0) & (concat_df[conditions[1]]==condition1) & (concat_df[conditions[2]]==condition2) & (concat_df[conditions[3]]==condition3)]
                            # number of go trials
                            if ('stop' in exp_id) and ('SS:go' in condition_name):
                                countGoTrials = str('countGoTrials_'+condition_name)
                                qa_df.at[worker_id,countGoTrials] = len(df_subj)  #len(df_subj): all Go trials in one of the other condition's levels
                            # mean response times
                            mrt = str('m(rt)_'+condition_name)
                            if len(df_subj['rt'][(df_subj['correct_trial']==1)])==0:
        #                         print(worker_id + ' had 0 correct trial in ' + condition_name + ' in ' + exp_id)
                                qa_df.at[worker_id,mrt] = np.nan
                            else:
                                mean_rt = round(stats.mean(df_subj['rt'][(df_subj['correct_trial']==1)]),2)
                                qa_df.at[worker_id,mrt] = mean_rt
                            # sd response times
                            sdrt = str('sd(rt)_'+condition_name)
                            if len(df_subj['rt'][(df_subj['correct_trial']==1)])<2:
        #                         print(worker_id + ' had <2 correct_trial in ' + condition_name + ' in ' + exp_id)
                                qa_df.at[worker_id,sdrt] = np.nan
                            else:
                                sd_rt = round(stats.stdev(df_subj['rt'][(df_subj['correct_trial']==1)]),2)
                                qa_df.at[worker_id,sdrt] = sd_rt
                            # accuracy rate, i.e., count of correct responses
                            if len(df_subj) !=0:
                                accuracy = round(len(df_subj[df_subj['correct_trial']==1])/len(df_subj),2)
                                acc = str('acc_'+condition_name)
                                qa_df.at[worker_id,acc] = accuracy
#                             else:
#                                 qa_df.at[worker_id,acc] = np.nan
                            if '_no' in exp_id and condition0 == 'nogo' or condition1 == 'nogo' or condition2 == 'nogo' or condition3 == 'nogo' and 'go_nogo_condition' in df_subj.columns:
                                df_subj_nogo = df_subj[(df_subj['go_nogo_condition']=='nogo')]
                                # nogofail rate
                                if len(df_subj_nogo) !=0:
                                    nogofail = round(len(df_subj_nogo[df_subj_nogo['correct_trial']==0])/len(df_subj_nogo),2)
                                    nogo_fail = str('nogofail_rate_'+condition_name)
                                    qa_df.at[worker_id,nogo_fail] = nogofail
                                else:
                                    qa_df.at[worker_id,nogo_fail] = np.nan
                                #m(rt_nogofail)
                                mrt_nogofail = str('m(nogofail_rt)_'+condition_name)
                                if len(df_subj_nogo['rt'][(df_subj_nogo['correct_trial']==0)])==0:
        #                             print(worker_id + ' had 0 nogofail trial in ' + condition_name + ' in ' + exp_id)
                                    qa_df.at[worker_id,mrt_nogofail] = np.nan
                                else:
                                    mrtfail = round(stats.mean(df_subj_nogo['rt'][(df_subj_nogo['correct_trial']==0)]),2)
                                    qa_df.at[worker_id,mrt_nogofail] = mrtfail
                                #sd(rt_nogofail)
                                sdrt_nogofail = str('sd(nogofail_rt)_'+condition_name)
                                if len(df_subj_nogo['rt'][(df_subj_nogo['correct_trial']==0)])<2:
        #                             print(worker_id + ' had <2 nogofail trial in ' + condition_name + ' in ' + exp_id)
                                    qa_df.at[worker_id,sdrt_nogofail] = np.nan
                                else:
                                    sdrtfail = round(stats.stdev(df_subj_nogo['rt'][(df_subj_nogo['correct_trial']==0)]),2)
                                    qa_df.at[worker_id,sdrt_nogofail] = sdrtfail
                            if 'stop' in exp_id and condition0 == 'stop' or condition1 == 'stop' or condition2 == 'stop' or condition3 == 'stop' and 'stop_signal_condition' in df_subj.columns and 'SS_delay' in df_subj.columns:
                                df_subj_stop = df_subj[(df_subj['stop_signal_condition']=='stop')]
                                # ssd
                                if len(df_subj_stop['SS_delay']) >1:
                                    ssd = round(stats.mean(df_subj_stop['SS_delay']),2)
                                    SS_delay = str('ssd_'+condition_name)
                                    qa_df.at[worker_id,SS_delay] = ssd
                                else:
                                    qa_df.at[worker_id,SS_delay] = np.nan
                                # stopfail rate
                                if len(df_subj_stop) !=0:
                                    stopfail = round(len(df_subj_stop[df_subj_stop['correct_trial']==0])/len(df_subj_stop),2)
                                    stop_fail = str('stopfail_rate_'+condition_name)
                                    qa_df.at[worker_id,stop_fail] = stopfail
                                else:
                                    qa_df.at[worker_id,stop_fail] = np.nan
                                #ssrt
                                SSRT = str('ssrt_'+condition_name)
                                rank = int(qa_df.at[worker_id,[i for i in qa_df.columns if ('countGoTrials_' in i) & (no_stop_condition_name in i) & ('SS:go' in i)][0]]*stopfail)
                                if rank >0:
                                    nthRT = list(concat_df['GoRTWithReplacement'][(concat_df['worker_id']==worker_id) & (concat_df[conditions[0]]==condition0) & (concat_df[conditions[1]]==condition1) & (concat_df[conditions[2]]==condition2) & (concat_df['GoRTWithReplacementRank']==rank)])[0]
                                    ssrt = nthRT - ssd
                                    qa_df.at[worker_id,SSRT] = round(ssrt,2)
                                else:
                                    qa_df.at[worker_id,SSRT] = 'rank <0'
                                #m(rt_stopfail)
                                mrt_stopfail = str('m(stopfail_rt)_'+condition_name)
                                if len(df_subj_stop['rt'][(df_subj_stop['correct_trial']==0)])==0:
        #                             print(worker_id + ' had 0 stopfail trial in ' + condition_name + ' in ' + exp_id)
                                    qa_df.at[worker_id,mrt_stopfail] = np.nan
                                else:
                                    mrtfail = round(stats.mean(df_subj_stop['rt'][(df_subj_stop['correct_trial']==0)]),2)
                                    qa_df.at[worker_id,mrt_stopfail] = mrtfail
                                #sd(rt_stopfail)
                                sdrt_stopfail = str('sd(stopfail_rt)_'+condition_name)
                                if len(df_subj_stop['rt'][(df_subj_stop['correct_trial']==0)])<2:
        #                             print(worker_id + ' had <2 stopfail trial in ' + condition_name + ' in ' + exp_id)
                                    qa_df.at[worker_id,sdrt_stopfail] = np.nan
                                else:
                                    sdrtfail = round(stats.stdev(df_subj_stop['rt'][(df_subj_stop['correct_trial']==0)]),2)
                                    qa_df.at[worker_id,sdrt_stopfail] = sdrtfail
        #                             #acc_stopfail
        #                             accfail = round(len(df_subj_stop[df_subj_stop['correct_trial']==0])/len(df_subj_stop)*100,2)
        #                             accrt_stopfail = str('acc(stopfail)_)_'+condition_name)
        #                             qa_df.at[worker_id,accrt_stopfail] = accfail
        #                     else:
        #                         print('stop_signal_condition and SS_delay NOT in '+exp_id+'.columns')
    qa_df.drop([i for i in qa_df.columns if ('countGoTrials' in i) & ('SS:stop' in i)], axis=1, inplace=True, errors='ignore')
    qa_df.drop([i for i in qa_df.columns if ('TASK:switch' in i) & ('CUE:stay' in i)], axis=1, inplace=True, errors='ignore')
    qa_df.drop([i for i in qa_df.columns if (('m(rt)' in i)|('sd(rt)' in i)|('acc_' in i)) & (('SS:stop' in i)|('GNG:nogo' in i))], axis=1, inplace=True, errors='ignore')
    if ('flanker_with_cued_task_switching' in exp_id) or ('shape_matching_with_cued_task_switching' in exp_id):
        qa_df.columns = qa_df.columns.str.replace(r'switch_new','switch')
#     qa_df.drop([i for i in qa_df.columns if 'DF:pos' in i], axis=1, inplace=True, errors='ignore')
    qa_df.drop([i for i in qa_df.columns if ('SSS' in i) | ('DSD' in i)], axis=1, inplace=True, errors='ignore')
    if not os.path.exists(concat_dir + 'QA/explore'): os.makedirs(concat_dir + 'QA/explore')
    qa_df.loc['mean'] = round(qa_df.mean(),2)
#     qa_df.to_csv(concat_dir + 'QA/explore/' + exp_id + '_qa.csv')
    qa_df.to_csv('~/Documents/Networking_Pilot/explore_n33/QA/' + exp_id + '_qa.csv')

In [ ]:
###concat all main_DVs tables to one file (after a couple of cleanings-up)
concat_mainDVs = pd.DataFrame()
for main_DVs_file in glob('/Users/tonyb/Documents/Networking_Pilot/explore_n33/main_DVs_analyses/csv_copy/*'):
    DVs_filename = os.path.basename(main_DVs_file)
    DVs_df = pd.read_csv(main_DVs_file)
    #make col names consistent
    # list of (old, new)
    #for i in list, replace i[0] with i[1]
    DVs_df.columns = DVs_df.columns.str.replace('ts','TS')
    DVs_df.columns = DVs_df.columns.str.replace('TScost','TS-cost')
    DVs_df.columns = DVs_df.columns.str.replace('FE','flanker')
    DVs_df.columns = DVs_df.columns.str.replace('fe','flanker')
    DVs_df.columns = DVs_df.columns.str.replace('df','DF')
    DVs_df.columns = DVs_df.columns.str.replace('Df','DF')
    DVs_df.columns = DVs_df.columns.str.replace('commission','GNG-commit')
    DVs_df.columns = DVs_df.columns.str.replace('ssrt','SSRT')
    DVs_df.columns = DVs_df.columns.str.replace('Unnamed: 0','explore_id')
    #specify TS-cost from predictable_ or cued_task-switching
    if 'predict' in DVs_filename and 'cued' not in DVs_filename:
        DVs_df.columns = DVs_df.columns.str.replace('TS','predict-TS')
    elif 'cued' in DVs_filename and 'predict' not in DVs_filename:
        DVs_df.columns = DVs_df.columns.str.replace('TS','cued-TS')
    #add the other task name to DC_ and DE_ columns
    task1 = DVs_df.columns[1]
    task2 = DVs_df.columns[2]
    DVs_df.columns = DVs_df.columns.str.replace('DC_%s' %task1, 'DC_%s_%s' %(task1,task2))
    DVs_df.columns = DVs_df.columns.str.replace('DE_%s' %task1, 'DE_%s_%s' %(task1,task2))
    DVs_df.columns = DVs_df.columns.str.replace('DC_%s' %task2, 'DC_%s_%s' %(task2,task1))
    DVs_df.columns = DVs_df.columns.str.replace('DE_%s' %task2, 'DE_%s_%s' %(task2,task1))
    #write main_DVs_analyses with workerId col and mean row still
    DVs_df.to_csv('/Users/tonyb/Documents/Networking_Pilot/explore_n33/main_DVs_analyses/csv/%s' %DVs_filename, index=False)
    #concat all main_DVs_analyses after removing the workerId col and the mean row
    concat_mainDVs = pd.concat([concat_mainDVs , DVs_df.iloc[:-1 , 1:]], axis=1)
#remove duplicated cols by column name
concat_mainDVs = concat_mainDVs.loc[:,~concat_mainDVs.columns.duplicated()]
#reorder concat df to be clustered by tasks then write
single_cols = [i for i in concat_mainDVs.columns if ('DC' not in i) and ('DE' not in i)]
single_cols = sorted(single_cols,key=str.casefold)
cued_cols = [i for i in concat_mainDVs.columns if (('cued-TS-cost' in i)&(i in single_cols)) or ('DC_cued' in i) or ('DE_cued' in i)]
DF_cols = [i for i in concat_mainDVs.columns if (('DF' in i)&(i in single_cols)) or ('DC_DF' in i) or ('DE_DF' in i)]
flanker_cols = [i for i in concat_mainDVs.columns if (('flanker' in i)&(i in single_cols)) or ('DC_flanker' in i) or ('DE_flanker' in i)]
GNG_cols = [i for i in concat_mainDVs.columns if (('GNG-commit' in i)&(i in single_cols)) or ('DC_GNG-commit' in i) or ('DE_GNG-commit' in i)]
nback_cols = [i for i in concat_mainDVs.columns if (('nback' in i)&(i in single_cols)) or ('DC_nback' in i) or ('DE_nback' in i)]
predict_cols = [i for i in concat_mainDVs.columns if (('predict-TS-cost' in i)&(i in single_cols)) or ('DC_predict' in i) or ('DE_predict' in i)]
shape_cols = [i for i in concat_mainDVs.columns if (('shape' in i)&(i in single_cols)) or ('DC_shape' in i) or ('DE_shape' in i)]
stop_cols = [i for i in concat_mainDVs.columns if (('SSRT' in i)&(i in single_cols)) or ('DC_SSRT' in i) or ('DE_SSRT' in i)]
cued_cols_df = concat_mainDVs.loc[:,concat_mainDVs.columns.isin(cued_cols)]
DF_cols_df = concat_mainDVs.loc[:,concat_mainDVs.columns.isin(DF_cols)]
flanker_cols_df = concat_mainDVs.loc[:,concat_mainDVs.columns.isin(flanker_cols)]
GNG_cols_df = concat_mainDVs.loc[:,concat_mainDVs.columns.isin(GNG_cols)]
nback_cols_df = concat_mainDVs.loc[:,concat_mainDVs.columns.isin(nback_cols)]
predict_cols_df = concat_mainDVs.loc[:,concat_mainDVs.columns.isin(predict_cols)]
shape_cols_df = concat_mainDVs.loc[:,concat_mainDVs.columns.isin(shape_cols)]
stop_cols_df = concat_mainDVs.loc[:,concat_mainDVs.columns.isin(stop_cols)]
concat_mainDVs = pd.concat([cued_cols_df,DF_cols_df,flanker_cols_df,GNG_cols_df,
                            nback_cols_df,predict_cols_df,shape_cols_df,stop_cols_df],axis=1)
#write concat df
concat_mainDVs.to_csv('/Users/tonyb/Documents/Networking_Pilot/explore_n33/main_DVs_analyses/concat/concat_mainDVs.csv', index=False)

In [ ]:
concat_mainDVs_corr = round(concat_mainDVs.corr(),3)
concat_mainDVs_corr.to_csv('/Users/tonyb/Documents/Networking_Pilot/explore_n33/main_DVs_analyses/concat/concat_mainDVs_corr_index=False.csv', index=False)

In [ ]:
# heatmap using matplot
# df = pd.read_csv('/Users/tonyb/Documents/Networking_Pilot/explore_n33/main_DVs_analyses/concat/concat_mainDVs.csv')#.iloc[:,105:len(concat_mainDVs.columns)+1]
# f = plt.figure(figsize=(130, 130), facecolor='w') #facecolor is background color
# plt.matshow(df.corr(), cmap=cm.seismic, fignum=f.number, vmin=-1, vmax=1)
# xticks = list(range(df.shape[1]))
# xticks.insert(df.shape[1]+1,df.shape[1]-0.5)
# yticks = list(range(df.shape[1]))
# yticks.insert(0,-0.5)
# yticks.insert(df.shape[1]+1,df.shape[1]-0.5)
# plt.xticks(xticks, df.columns, fontsize=15, rotation=80)
# plt.yticks(yticks, df.columns.insert(0,'').insert(len(df.columns)+1,''), fontsize=15)
# for y in range(df.shape[0]):
#     for x in range(df.shape[1]):
#         plt.text(x + 0.5, y + 0.5, '%.3f' % data.iloc[y, x],
#                  horizontalalignment='center',
#                  verticalalignment='center',
#                  )
# cb = plt.colorbar(shrink=0.5, aspect=30, fraction=.12, pad=.02)
# cb.ax.tick_params(labelsize=60)
# plt.title('Main-DVs Correlation Matrix', fontsize=80);

#corr data with (1) full index and col names; (2) 1's on diagonal replaced by means of corresponding raw data
annotData = pd.read_csv(main_DVs_concat_dir + 'concat_mainDVs_corr_index=False.csv')
annotData.index = annotData.columns
for i in annotData.index:
    for j in annotData.columns:
        if i==j:
            annotData.loc[i,j] = concat_mainDVs[j].mean()

#data for drawing heatmap
data = pd.read_csv(main_DVs_concat_dir + 'concat_mainDVs_corr_index=False.csv') 
column_labels = data.columns
row_labels = data.columns

#set up background (fig) and main map (ax)
fig, ax = plt.subplots(figsize=(130,130), facecolor='beige')
ax.set_title('All DV variants Corr Matrix',size=80)
heatmap = ax.pcolor(data, cmap=cm.seismic, vmin=-1, vmax=1)
 
# Put the major ticks at the middle of each cell
ax.set_xticks(np.arange(0.5,data.shape[1],1), minor=False)
ax.set_yticks(np.arange(0.5,data.shape[0],1), minor=False)

for y in range(data.shape[0]):
    for x in range(data.shape[1]):
        text = ax.text(x + 0.5, y + 0.5, '%.3f' % annotData.iloc[y, x],
                 horizontalalignment='center',
                 verticalalignment='center',
                 size=11)
 
# Want a more natural, table-like display
ax.axis('square')
ax.invert_yaxis()
ax.xaxis.tick_top()
 
ax.set_xticklabels(row_labels, minor=False, rotation=80, size=15)
ax.set_yticklabels(column_labels, minor=False, size=15)

# color bar
cb = plt.colorbar(heatmap,shrink=0.5, aspect=30, fraction=.12, pad=.02)
cb.ax.tick_params(labelsize=60)
plt.show()

In [ ]:
taskByTask_meanCorr_matrix = pd.DataFrame(index=single_cols,columns=single_cols)
for i in single_cols:
    start_index = concat_mainDVs_corr.index.get_loc(i)
    for j in single_cols:
        start_col = concat_mainDVs_corr.columns.get_loc(j)
        #get the concat df
        taskByTask_corrDf = concat_mainDVs_corr.iloc[start_index:start_index+15 , start_col:start_col+15]
        if not os.path.exists(main_DVs_concat_dir + 'taskByTask_corr/'): os.makedirs(main_DVs_concat_dir + 'taskByTask_corr/')
        taskByTask_corrDf.to_csv(main_DVs_concat_dir + 'taskByTask_corr/%s_by_%s.csv' %(i,j))
        #exclude 1's on the diagonal in correlation of the same 15 variants (i.e. of the same main_DV)
        if i==j:
            for a in range(taskByTask_corrDf.shape[0]):
                for b in range(taskByTask_corrDf.shape[1]):
                    if a==b:
                        taskByTask_corrDf.iloc[a,b] = np.nan
        #get average of all 15x15 corr numbers
        taskByTask_corrDf_mean = round(taskByTask_corrDf.iloc[:,:].mean().mean(),3)
        taskByTask_meanCorr_matrix.loc[i,j] = taskByTask_corrDf_mean
taskByTask_meanCorr_matrix.to_csv(main_DVs_concat_dir + 'taskByTask_meanCorr_matrix.csv', index=False)

In [ ]:
df = pd.read_csv(main_DVs_concat_dir+'concat_8x8_corr.csv')

fig = plt.figure(figsize=(8,8),facecolor='beige')
plt.title('Mean Correlation Matrix',size=14,pad=15)
plt.pcolor(df,cmap=cm.seismic,vmin=-1,vmax=1)
plt.yticks(np.arange(0.5, len(concat_8x8_corr.index), 1), concat_8x8_corr.index, size=11)
plt.xticks(np.arange(0.5, len(df.columns), 1), df.columns, rotation=70, size=11)
plt.axis('square')
for y in range(df.shape[0]):
    for x in range(df.shape[1]):
        plt.text(x + 0.5, y + 0.5, '%.3f' % df.iloc[y, x],
                 horizontalalignment='center',
                 verticalalignment='center',
                 size=11)
cb = plt.colorbar(shrink=0.8, aspect=12.5, fraction=.12, pad=.03)
cb.ax.tick_params(labelsize=11)
plt.show()

In [ ]:
data = pd.read_csv(main_DVs_concat_dir + 'taskByTask_meanCorr_matrix.csv')

column_labels = data.columns
row_labels = concat_8x8_corr.index
 
fig, ax = plt.subplots(figsize=(7,7), facecolor='beige')
ax.set_title('Mean Correlation Matrix',size=15,pad=10)
heatmap = ax.pcolor(data, cmap=cm.seismic, vmin=-1, vmax=1)
 
# Put the major ticks at the middle of each cell
ax.set_xticks(np.arange(0.5,data.shape[1],1), minor=False)
ax.set_yticks(np.arange(0.5,data.shape[0],1), minor=False)

for y in range(data.shape[0]):
    for x in range(data.shape[1]):
        text = ax.text(x + 0.5, y + 0.5, '%.3f' % data.iloc[y, x],
                 horizontalalignment='center',
                 verticalalignment='center',
                 size=11)
 
# Want a more natural, table-like display
ax.axis('square')
ax.invert_yaxis()
ax.xaxis.tick_top()
 
ax.set_xticklabels(row_labels, minor=False, rotation=70, size=11)
ax.set_yticklabels(column_labels, minor=False, size=11)

# color bar
cb = plt.colorbar(heatmap,shrink=0.875, aspect=10.5, fraction=.12, pad=.03)
cb.ax.tick_params(labelsize=11)
plt.show()

In [ ]:
### check for all-empty columns
for qa_file in glob('/Users/tonyb/Desktop/all_data/QA/explore/*'):
    qa_df = pd.read_csv(qa_file)
    for i in qa_df.columns:
        if qa_df[i].isnull().all():
            print(i)

In [ ]:
### checking if any two m(rt) columns are identical and not equal to -1, which is the value for no response made
for test_file in glob('/Users/tonyb/Desktop/all_data/QA/explore/*'):
    test = pd.read_csv(test_file)
#     test.columns = test.columns.droplevel(0) #if there are multiindex in columns
    for i in range(len(test.columns)-1):
        for j in range(i+1,len(test.columns)):
            count = 0
            for row in range(len(test)):
                if test.iloc[row,i] == test.iloc[row,j] or np.isnan(test.iloc[row,i]) and np.isnan(test.iloc[row,j]): 
                    count +=1
            if count == len(test): print('%s == %s' %(test.columns[i],test.columns[j]))

In [ ]:
### DEPRECATED!!! Kept because I'm weak...

#8x8 map where each of the 8 diagonal cells is the average off of a 15x15 corr matrix among all 15 variants of each task (after removing the fifteen 1's), 
#and off-diagonal cells are averages from 30x30 matrices among 15 variants of 1 task and 15 variants of one of the other 7 tasks (after removing the fifteen 1's).
task_dfs_list = [cued_cols_df,DF_cols_df,flanker_cols_df,GNG_cols_df,
                 nback_cols_df,predict_cols_df,shape_cols_df,stop_cols_df]
main_DVs_concat_dir = '/Users/tonyb/Documents/Networking_Pilot/explore_n33/main_DVs_analyses/concat/'
if not os.path.exists(main_DVs_concat_dir + 'task_x_task/'):
    os.makedirs(main_DVs_concat_dir + 'task_x_task/')
if not os.path.exists(main_DVs_concat_dir + 'task_x_task_corr/'):
    os.makedirs(main_DVs_concat_dir + 'task_x_task_corr/')
#getting a combination of 2 out of 8 tasks with replacement so that a pair of one task with itself is also included
combination_tasks = list(itertools.combinations_with_replacement(range(len(task_dfs_list)),2))
#making 8x8 table
concat_8x8_corr = pd.DataFrame(index=sorted(single_cols, key=str.casefold), columns=sorted(single_cols, key=str.casefold))
for combination in combination_tasks:
    task1_index = combination[0]
    task2_index = combination[1]
    if task1_index == task2_index: #combination of a task with itself, e.g., cued with cued
        concat_combination_df = task_dfs_list[task1_index]
        file_name = concat_combination_df.columns[concat_combination_df.columns.isin(single_cols)][0] #there is only one column in concat_combination_df that matches with one element of the single_cols list, so wherever that is will be used for the file name
    else:
        
        concat_combination_df = pd.concat([task_dfs_list[task1_index], task_dfs_list[task2_index]], axis=1)
        task1_name = concat_combination_df.columns[concat_combination_df.columns.isin(single_cols)][0]
        task2_name = concat_combination_df.columns[concat_combination_df.columns.isin(single_cols)][1]
        file_name = str(task1_name + '_by_' + task2_name)
    concat_combination_df.to_csv(main_DVs_concat_dir + 'task_x_task/' + file_name + '.csv', index=False)
    concat_combin_corr_df = concat_combination_df.corr().replace(1,np.nan)
    concat_combin_corr_df.to_csv(main_DVs_concat_dir + 'task_x_task_corr/' + file_name + '_corr.csv', index=False)
    #calculate average of all corr in the current task_x_task df
    concat_combin_corr_mean = round(concat_combin_corr_df.iloc[:,:].mean().mean(),3)
    if task1_index == task2_index:
        concat_8x8_corr.loc[file_name,file_name] = concat_combin_corr_mean
    else:
        concat_8x8_corr.loc[task1_name,task2_name] = concat_combin_corr_mean
        concat_8x8_corr.loc[task2_name,task1_name] = concat_combin_corr_mean
concat_8x8_corr.to_csv(main_DVs_concat_dir + 'concat_8x8_corr.csv',index=False)